this scripts will filter and regress out flagged timepoints and motion regressors; a side-by-side comparison of this to SPM denoised data reveals that they are both doing fairly similar things which gives me great confidence that I did this correctly!

## load packages

In [1]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os 

import numpy as np
from nipype.interfaces import afni
import nibabel as nib
from nilearn.input_data import NiftiMasker,  MultiNiftiMasker
from nilearn.masking import intersect_masks
import seaborn as sns 
import pandas as pd
import matplotlib.pyplot as plt
from pandas import *

%autosave 5
%matplotlib inline

/Users/alexbarnett/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Autosaving every 5 seconds


## set directories


In [2]:
data_dir = '/Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_output_test/fmriprep/'
analysis_dir = '/Users/alexbarnett/Documents/TTTvsPMAT/fMRI/SRM/'
subjlist = pd.read_excel(analysis_dir+'subjlist.xlsx')


## loop through subjects and denoise SRM movie

In [5]:
for i, sub in subjlist.iterrows():
    #load in confounds and select relevant columns
    confound_full = pd.read_csv(data_dir+sub['subj_id']+'/'+sub['ses_num']+'/func/'+sub['subj_id']+'_'+sub['ses_num']+'_task-srmmovie_run-1_desc-confounds_regressors.tsv', sep='\t')
    index = ['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']
    confound_motion_outliers = confound_full.filter(regex="motion_outlier")
    confound_motion = confound_full[index]
    confound_select = pd.concat([confound_motion_outliers,confound_motion],axis = 1)
    confound_select.to_csv(data_dir+sub['subj_id']+'/'+sub['ses_num']+'/func/'+sub['subj_id']+'_'+sub['ses_num']+'_task-srmmovie_run-1_desc-confounds_regressors_select.1D', header=False, index= False)
    
    #perform filtering and denoising
    tproject = afni.TProject()
    tproject.inputs.in_file = data_dir+sub['subj_id']+'/'+sub['ses_num']+'/func/'+sub['subj_id']+'_'+sub['ses_num']+'_task-srmmovie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
    tproject.inputs.bandpass = (0.005, 99999)
    tproject.inputs.ort = (data_dir+sub['subj_id']+'/'+sub['ses_num']+'/func/'+sub['subj_id']+'_'+sub['ses_num']+'_task-srmmovie_run-1_desc-confounds_regressors_select.1D')
    tproject.inputs.automask = False
    tproject.inputs.out_file = analysis_dir+'confound_corrected_data/'+sub['subj_id']+'_'+sub['ses_num']+'_task-srmmovie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised.nii.gz'
    tproject.inputs.norm = False
    res = tproject.run()  
 

200322-14:07:26,611 nipype.interface INFO:
	 stderr 2020-03-22T14:07:26.611751:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200322-14:07:26,613 nipype.interface INFO:
	 stderr 2020-03-22T14:07:26.611751:++ Authored by: Cox the Algebraic (Linear)
200322-14:07:26,613 nipype.interface INFO:
	 stderr 2020-03-22T14:07:26.613962:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_output_test/fmriprep/sub-000/ses-02/func/sub-000_ses-02_task-srmmovie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
200322-14:07:26,614 nipype.interface INFO:
	 stderr 2020-03-22T14:07:26.613962:     Warnings of this type will be muted for this session.
200322-14:07:26,615 nipype.interface INFO:
	 stderr 2020-03-22T14:07:26.613962:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
200322-14:07:28,550 nipype.interface INFO:
	 stderr 2020-03-22T14:07:28.550742:++ Setting up regressors
200322-14:07:28,551 nipy

200322-14:07:49,827 nipype.interface INFO:
	 stderr 2020-03-22T14:07:49.826852:** ERROR: dataset NOT written to disk!
200322-14:07:49,828 nipype.interface INFO:
	 stderr 2020-03-22T14:07:49.826852:++ ===== clock time = 7s 469ms
200322-14:07:50,10 nipype.interface INFO:
	 stderr 2020-03-22T14:07:50.010703:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200322-14:07:50,12 nipype.interface INFO:
	 stderr 2020-03-22T14:07:50.010703:++ Authored by: Cox the Algebraic (Linear)
200322-14:07:50,14 nipype.interface INFO:
	 stderr 2020-03-22T14:07:50.014226:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_output_test/fmriprep/sub-103/ses-01/func/sub-103_ses-01_task-srmmovie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
200322-14:07:50,14 nipype.interface INFO:
	 stderr 2020-03-22T14:07:50.014226:     Warnings of this type will be muted for this session.
200322-14:07:50,15 nipype.interface INFO:
	 stderr 20

200322-14:08:13,831 nipype.interface INFO:
	 stderr 2020-03-22T14:08:13.830990:** ERROR: output dataset name 'sub-105_ses-01_task-srmmovie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised.nii.gz' conflicts with existing file
200322-14:08:13,831 nipype.interface INFO:
	 stderr 2020-03-22T14:08:13.830990:** ERROR: dataset NOT written to disk!
200322-14:08:13,832 nipype.interface INFO:
	 stderr 2020-03-22T14:08:13.830990:++ ===== clock time = 8s 344ms
200322-14:08:14,20 nipype.interface INFO:
	 stderr 2020-03-22T14:08:14.020583:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200322-14:08:14,22 nipype.interface INFO:
	 stderr 2020-03-22T14:08:14.020583:++ Authored by: Cox the Algebraic (Linear)
200322-14:08:14,23 nipype.interface INFO:
	 stderr 2020-03-22T14:08:14.023338:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_output_test/fmriprep/sub-106/ses-02/func/sub-106_ses-02_task-srmmovie_run-1_space-MNI152NLin2

200322-14:08:33,493 nipype.interface INFO:
	 stderr 2020-03-22T14:08:33.493915:++ Starting project-orization
200322-14:08:34,243 nipype.interface INFO:
	 stderr 2020-03-22T14:08:34.243100:++ Convert results to output dataset
200322-14:08:37,243 nipype.interface INFO:
	 stderr 2020-03-22T14:08:37.243572:** ERROR: output dataset name 'sub-109_ses-02_task-srmmovie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised.nii.gz' conflicts with existing file
200322-14:08:37,244 nipype.interface INFO:
	 stderr 2020-03-22T14:08:37.243572:** ERROR: dataset NOT written to disk!
200322-14:08:37,245 nipype.interface INFO:
	 stderr 2020-03-22T14:08:37.243572:++ ===== clock time = 7s 616ms
200322-14:08:37,417 nipype.interface INFO:
	 stderr 2020-03-22T14:08:37.417741:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200322-14:08:37,420 nipype.interface INFO:
	 stderr 2020-03-22T14:08:37.417741:++ Authored by: Cox the Algebraic (Linear)
200322-14:08:37,420 nipype.interface INFO:
	 st

200322-14:08:55,123 nipype.interface INFO:
	 stderr 2020-03-22T14:08:55.122230:++ Loading dataset
200322-14:08:57,6 nipype.interface INFO:
	 stderr 2020-03-22T14:08:57.006954:++ Starting project-orization
200322-14:08:57,810 nipype.interface INFO:
	 stderr 2020-03-22T14:08:57.810399:++ Convert results to output dataset
200322-14:09:14,312 nipype.interface INFO:
	 stderr 2020-03-22T14:09:14.312561:++ Output dataset /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/SRM/confound_corrected_data/sub-117_ses-02_task-srmmovie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised.nii.gz
200322-14:09:14,313 nipype.interface INFO:
	 stderr 2020-03-22T14:09:14.312561:++ ===== clock time = 21s 375ms
200322-14:09:14,549 nipype.interface INFO:
	 stderr 2020-03-22T14:09:14.549552:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200322-14:09:14,551 nipype.interface INFO:
	 stderr 2020-03-22T14:09:14.549552:++ Authored by: Cox the Algebraic (Linear)
200322-14:09:14,561 nipype.interface IN

200322-14:10:22,286 nipype.interface INFO:
	 stderr 2020-03-22T14:10:22.286110:++ Output dataset /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/SRM/confound_corrected_data/sub-121_ses-02_task-srmmovie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised.nii.gz
200322-14:10:22,286 nipype.interface INFO:
	 stderr 2020-03-22T14:10:22.286110:++ ===== clock time = 22s 352ms
200322-14:10:22,500 nipype.interface INFO:
	 stderr 2020-03-22T14:10:22.500101:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200322-14:10:22,501 nipype.interface INFO:
	 stderr 2020-03-22T14:10:22.500101:++ Authored by: Cox the Algebraic (Linear)
200322-14:10:22,525 nipype.interface INFO:
	 stderr 2020-03-22T14:10:22.525947:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_output_test/fmriprep/sub-123/ses-02/func/sub-123_ses-02_task-srmmovie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
200322-14:10:22,529 nipype.interface INF

## loop through subjects and denoise barmaid

In [6]:
for i, sub in subjlist.iterrows():
    #load in confounds and select relevant columns
    confound_full = pd.read_csv(data_dir+sub['subj_id']+'/'+sub['barmaid_ses']+'/func/'+sub['subj_id']+'_'+sub['barmaid_ses']+'_task-movieviewing_'+ sub['barmaid_place']+ '_desc-confounds_regressors.tsv', sep='\t')
    index = ['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']
    confound_motion_outliers = confound_full.filter(regex="motion_outlier")
    confound_motion = confound_full[index]
    confound_select = pd.concat([confound_motion_outliers,confound_motion],axis = 1)
    confound_select.to_csv(data_dir+sub['subj_id']+'/'+sub['barmaid_ses']+'/func/'+sub['subj_id']+'_'+sub['barmaid_ses']+'_task-movieviewing_'+ sub['barmaid_place']+ '_desc-confounds_regressors_select.1D', header=False, index= False)
    
    #perform filtering and denoising
    tproject = afni.TProject()
    tproject.inputs.in_file = data_dir+sub['subj_id']+'/'+sub['barmaid_ses']+'/func/'+sub['subj_id']+'_'+sub['barmaid_ses']+'_task-movieviewing_'+ sub['barmaid_place']+'_space-MNI152NLin2009cAsym_desc-preproc_bold.nii'
    tproject.inputs.bandpass = (0.005, 99999)
    tproject.inputs.ort = (data_dir+sub['subj_id']+'/'+sub['barmaid_ses']+'/func/'+sub['subj_id']+'_'+sub['barmaid_ses']+'_task-movieviewing_'+ sub['barmaid_place']+ '_desc-confounds_regressors_select.1D')
    tproject.inputs.automask = False
    tproject.inputs.out_file = analysis_dir+'confound_corrected_data/'+sub['subj_id']+'_'+sub['barmaid_ses']+'_task-movieviewing_'+ sub['barmaid_place']+'_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised.nii.gz'
    tproject.inputs.norm = False
    res = tproject.run()  


200323-09:13:15,340 nipype.interface INFO:
	 stderr 2020-03-23T09:13:15.340641:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200323-09:13:15,342 nipype.interface INFO:
	 stderr 2020-03-23T09:13:15.340641:++ Authored by: Cox the Algebraic (Linear)
200323-09:13:15,343 nipype.interface INFO:
	 stderr 2020-03-23T09:13:15.343229:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_output_test/fmriprep/sub-000/ses-01/func/sub-000_ses-01_task-movieviewing_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii to FLOAT32
200323-09:13:15,344 nipype.interface INFO:
	 stderr 2020-03-23T09:13:15.343229:     Warnings of this type will be muted for this session.
200323-09:13:15,344 nipype.interface INFO:
	 stderr 2020-03-23T09:13:15.343229:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
200323-09:13:20,271 nipype.interface INFO:
	 stderr 2020-03-23T09:13:20.271887:++ Setting up regressors
200323-09:13:20,272 nip

200323-09:13:55,993 nipype.interface INFO:
	 stderr 2020-03-23T09:13:55.993311:++ Starting project-orization
200323-09:13:57,349 nipype.interface INFO:
	 stderr 2020-03-23T09:13:57.349908:++ Convert results to output dataset
200323-09:14:01,926 nipype.interface INFO:
	 stderr 2020-03-23T09:14:01.926788:** ERROR: output dataset name 'sub-102_ses-01_task-movieviewing_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised.nii.gz' conflicts with existing file
200323-09:14:01,927 nipype.interface INFO:
	 stderr 2020-03-23T09:14:01.926788:** ERROR: dataset NOT written to disk!
200323-09:14:01,927 nipype.interface INFO:
	 stderr 2020-03-23T09:14:01.926788:++ ===== clock time = 14s 709ms
200323-09:14:02,442 nipype.interface INFO:
	 stderr 2020-03-23T09:14:02.442253:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200323-09:14:02,444 nipype.interface INFO:
	 stderr 2020-03-23T09:14:02.442253:++ Authored by: Cox the Algebraic (Linear)
200323-09:14:02,463 nipype.interface INFO:

200323-09:14:38,883 nipype.interface INFO:
	 stderr 2020-03-23T09:14:38.883335:++ no -mask option ==> processing all 325325 voxels in dataset
200323-09:14:38,883 nipype.interface INFO:
	 stderr 2020-03-23T09:14:38.883335:++ Compute pseudo-inverse of fixed orts
200323-09:14:38,884 nipype.interface INFO:
	 stderr 2020-03-23T09:14:38.883335:++ Loading dataset
200323-09:14:38,884 nipype.interface INFO:
	 stderr 2020-03-23T09:14:38.883335:++ THD_dset_to_vectim: allocated 1093092000 bytes
200323-09:14:41,905 nipype.interface INFO:
	 stderr 2020-03-23T09:14:41.905146:*+ WARNING: 3dTproject input data :: 109972 vectors are constant
200323-09:14:41,960 nipype.interface INFO:
	 stderr 2020-03-23T09:14:41.960237:++ Starting project-orization
200323-09:14:43,526 nipype.interface INFO:
	 stderr 2020-03-23T09:14:43.526582:++ Convert results to output dataset
200323-09:14:48,102 nipype.interface INFO:
	 stderr 2020-03-23T09:14:48.101933:** ERROR: output dataset name 'sub-105_ses-02_task-movieviewing_

200323-09:15:16,238 nipype.interface INFO:
	 stderr 2020-03-23T09:15:16.238107:++ setting up stopband frequency mask
200323-09:15:16,239 nipype.interface INFO:
	 stderr 2020-03-23T09:15:16.238107: + Block #0: 840 time points -- 11 stopband regressors
200323-09:15:16,239 nipype.interface INFO:
	 stderr 2020-03-23T09:15:16.238107:++ 1 Blocks * 3 polynomials -- 3 polort regressors
200323-09:15:16,240 nipype.interface INFO:
	 stderr 2020-03-23T09:15:16.238107: + -- 6 other fixed ort regressors
200323-09:15:16,240 nipype.interface INFO:
	 stderr 2020-03-23T09:15:16.238107:++ 840 retained time points MINUS 20 regressors ==> 820 D.O.F. left
200323-09:15:16,241 nipype.interface INFO:
	 stderr 2020-03-23T09:15:16.241794:++ no -mask option ==> processing all 325325 voxels in dataset
200323-09:15:16,242 nipype.interface INFO:
	 stderr 2020-03-23T09:15:16.241794:++ Compute pseudo-inverse of fixed orts
200323-09:15:16,242 nipype.interface INFO:
	 stderr 2020-03-23T09:15:16.241794:++ Loading dataset

200323-09:16:32,748 nipype.interface INFO:
	 stderr 2020-03-23T09:16:32.746724:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
200323-09:16:37,616 nipype.interface INFO:
	 stderr 2020-03-23T09:16:37.616537:++ Setting up regressors
200323-09:16:37,617 nipype.interface INFO:
	 stderr 2020-03-23T09:16:37.616537:++ setting up stopband frequency mask
200323-09:16:37,617 nipype.interface INFO:
	 stderr 2020-03-23T09:16:37.616537: + Block #0: 840 time points -- 11 stopband regressors
200323-09:16:37,618 nipype.interface INFO:
	 stderr 2020-03-23T09:16:37.616537:++ 1 Blocks * 3 polynomials -- 3 polort regressors
200323-09:16:37,618 nipype.interface INFO:
	 stderr 2020-03-23T09:16:37.616537: + -- 10 other fixed ort regressors
200323-09:16:37,619 nipype.interface INFO:
	 stderr 2020-03-23T09:16:37.616537:++ 840 retained time points MINUS 24 regressors ==> 816 D.O.F. left
200323-09:16:37,620 nipype.interface INFO:
	 stderr 2020-03-23T09:16:37.620251:++ no -mask option ==> pr

200323-09:18:26,345 nipype.interface INFO:
	 stderr 2020-03-23T09:18:26.345254:++ Setting up regressors
200323-09:18:26,345 nipype.interface INFO:
	 stderr 2020-03-23T09:18:26.345254:++ setting up stopband frequency mask
200323-09:18:26,346 nipype.interface INFO:
	 stderr 2020-03-23T09:18:26.345254: + Block #0: 840 time points -- 11 stopband regressors
200323-09:18:26,346 nipype.interface INFO:
	 stderr 2020-03-23T09:18:26.345254:++ 1 Blocks * 3 polynomials -- 3 polort regressors
200323-09:18:26,347 nipype.interface INFO:
	 stderr 2020-03-23T09:18:26.345254: + -- 17 other fixed ort regressors
200323-09:18:26,348 nipype.interface INFO:
	 stderr 2020-03-23T09:18:26.345254:++ 840 retained time points MINUS 31 regressors ==> 809 D.O.F. left
200323-09:18:26,349 nipype.interface INFO:
	 stderr 2020-03-23T09:18:26.349054:++ no -mask option ==> processing all 325325 voxels in dataset
200323-09:18:26,349 nipype.interface INFO:
	 stderr 2020-03-23T09:18:26.349054:++ Compute pseudo-inverse of fix

## loop through subjects and denoise bluestreets

In [8]:
for i, sub in subjlist.iterrows():
    #load in confounds and select relevant columns
    confound_full = pd.read_csv(data_dir+sub['subj_id']+'/'+sub['bluestreets_ses']+'/func/'+sub['subj_id']+'_'+sub['bluestreets_ses']+'_task-movieviewing_'+ sub['bluestreets_place']+ '_desc-confounds_regressors.tsv', sep='\t')
    index = ['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']
    confound_motion_outliers = confound_full.filter(regex="motion_outlier")
    confound_motion = confound_full[index]
    confound_select = pd.concat([confound_motion_outliers,confound_motion],axis = 1)
    confound_select.to_csv(data_dir+sub['subj_id']+'/'+sub['bluestreets_ses']+'/func/'+sub['subj_id']+'_'+sub['bluestreets_ses']+'_task-movieviewing_'+ sub['bluestreets_place']+ '_desc-confounds_regressors_select.1D', header=False, index= False)
    
    #perform filtering and denoising
    tproject = afni.TProject()
    tproject.inputs.in_file = data_dir+sub['subj_id']+'/'+sub['bluestreets_ses']+'/func/'+sub['subj_id']+'_'+sub['bluestreets_ses']+'_task-movieviewing_'+ sub['bluestreets_place']+'_space-MNI152NLin2009cAsym_desc-preproc_bold.nii'
    tproject.inputs.bandpass = (0.005, 99999)
    tproject.inputs.ort = (data_dir+sub['subj_id']+'/'+sub['bluestreets_ses']+'/func/'+sub['subj_id']+'_'+sub['bluestreets_ses']+'_task-movieviewing_'+ sub['bluestreets_place']+ '_desc-confounds_regressors_select.1D')
    tproject.inputs.automask = False
    tproject.inputs.out_file = analysis_dir+'confound_corrected_data/'+sub['subj_id']+'_'+sub['bluestreets_ses']+'_task-movieviewing_'+ sub['bluestreets_place']+'_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised.nii.gz'
    tproject.inputs.norm = False
    res = tproject.run() 

200331-07:54:03,30 nipype.interface INFO:
	 stderr 2020-03-31T07:54:03.030295:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200331-07:54:03,32 nipype.interface INFO:
	 stderr 2020-03-31T07:54:03.030295:++ Authored by: Cox the Algebraic (Linear)
200331-07:54:03,79 nipype.interface INFO:
	 stderr 2020-03-31T07:54:03.079293:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_output_test/fmriprep/sub-000/ses-01/func/sub-000_ses-01_task-movieviewing_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii to FLOAT32
200331-07:54:03,79 nipype.interface INFO:
	 stderr 2020-03-31T07:54:03.079293:     Warnings of this type will be muted for this session.
200331-07:54:03,80 nipype.interface INFO:
	 stderr 2020-03-31T07:54:03.079293:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
200331-07:54:05,214 nipype.interface INFO:
	 stderr 2020-03-31T07:54:05.214019:++ Setting up regressors
200331-07:54:05,214 nipype.i

200331-07:54:33,843 nipype.interface INFO:
	 stderr 2020-03-31T07:54:33.843701:++ Convert results to output dataset
200331-07:54:38,339 nipype.interface INFO:
	 stderr 2020-03-31T07:54:38.339356:** ERROR: output dataset name 'sub-102_ses-01_task-movieviewing_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised.nii.gz' conflicts with existing file
200331-07:54:38,340 nipype.interface INFO:
	 stderr 2020-03-31T07:54:38.339356:** ERROR: dataset NOT written to disk!
200331-07:54:38,340 nipype.interface INFO:
	 stderr 2020-03-31T07:54:38.339356:++ ===== clock time = 12s 147ms
200331-07:54:38,572 nipype.interface INFO:
	 stderr 2020-03-31T07:54:38.572629:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200331-07:54:38,574 nipype.interface INFO:
	 stderr 2020-03-31T07:54:38.572629:++ Authored by: Cox the Algebraic (Linear)
200331-07:54:38,576 nipype.interface INFO:
	 stderr 2020-03-31T07:54:38.576033:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Doc

200331-07:55:03,226 nipype.interface INFO:
	 stderr 2020-03-31T07:55:03.225460:++ Compute pseudo-inverse of fixed orts
200331-07:55:03,226 nipype.interface INFO:
	 stderr 2020-03-31T07:55:03.225460:++ Loading dataset
200331-07:55:03,227 nipype.interface INFO:
	 stderr 2020-03-31T07:55:03.225460:++ THD_dset_to_vectim: allocated 1069668600 bytes
200331-07:55:06,308 nipype.interface INFO:
	 stderr 2020-03-31T07:55:06.308091:*+ WARNING: 3dTproject input data :: 111838 vectors are constant
200331-07:55:06,356 nipype.interface INFO:
	 stderr 2020-03-31T07:55:06.356581:++ Starting project-orization
200331-07:55:07,706 nipype.interface INFO:
	 stderr 2020-03-31T07:55:07.706703:++ Convert results to output dataset
200331-07:55:12,225 nipype.interface INFO:
	 stderr 2020-03-31T07:55:12.225393:** ERROR: output dataset name 'sub-105_ses-01_task-movieviewing_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised.nii.gz' conflicts with existing file
200331-07:55:12,226 nipype.interface INFO:
	 s

200331-07:55:35,333 nipype.interface INFO:
	 stderr 2020-03-31T07:55:35.330935:++ 819 retained time points MINUS 23 regressors ==> 796 D.O.F. left
200331-07:55:35,334 nipype.interface INFO:
	 stderr 2020-03-31T07:55:35.334283:++ no -mask option ==> processing all 325325 voxels in dataset
200331-07:55:35,335 nipype.interface INFO:
	 stderr 2020-03-31T07:55:35.334283:++ Compute pseudo-inverse of fixed orts
200331-07:55:35,335 nipype.interface INFO:
	 stderr 2020-03-31T07:55:35.334283:++ Loading dataset
200331-07:55:35,336 nipype.interface INFO:
	 stderr 2020-03-31T07:55:35.334283:++ THD_dset_to_vectim: allocated 1065764700 bytes
200331-07:55:38,335 nipype.interface INFO:
	 stderr 2020-03-31T07:55:38.335049:*+ WARNING: 3dTproject input data :: 114710 vectors are constant
200331-07:55:38,345 nipype.interface INFO:
	 stderr 2020-03-31T07:55:38.345010:++ Starting project-orization
200331-07:55:40,11 nipype.interface INFO:
	 stderr 2020-03-31T07:55:40.011168:++ Convert results to output datas

200331-07:56:52,348 nipype.interface INFO:
	 stderr 2020-03-31T07:56:52.347975:++ setting up stopband frequency mask
200331-07:56:52,349 nipype.interface INFO:
	 stderr 2020-03-31T07:56:52.347975: + Block #0: 816 time points -- 11 stopband regressors
200331-07:56:52,349 nipype.interface INFO:
	 stderr 2020-03-31T07:56:52.347975:++ 1 Blocks * 3 polynomials -- 3 polort regressors
200331-07:56:52,350 nipype.interface INFO:
	 stderr 2020-03-31T07:56:52.347975: + -- 7 other fixed ort regressors
200331-07:56:52,351 nipype.interface INFO:
	 stderr 2020-03-31T07:56:52.347975:++ 816 retained time points MINUS 21 regressors ==> 795 D.O.F. left
200331-07:56:52,352 nipype.interface INFO:
	 stderr 2020-03-31T07:56:52.352617:++ no -mask option ==> processing all 325325 voxels in dataset
200331-07:56:52,353 nipype.interface INFO:
	 stderr 2020-03-31T07:56:52.352617:++ Compute pseudo-inverse of fixed orts
200331-07:56:52,354 nipype.interface INFO:
	 stderr 2020-03-31T07:56:52.352617:++ Loading dataset

200331-07:58:32,228 nipype.interface INFO:
	 stderr 2020-03-31T07:58:32.228653:++ Setting up regressors
200331-07:58:32,229 nipype.interface INFO:
	 stderr 2020-03-31T07:58:32.228653:++ setting up stopband frequency mask
200331-07:58:32,229 nipype.interface INFO:
	 stderr 2020-03-31T07:58:32.228653: + Block #0: 819 time points -- 12 stopband regressors
200331-07:58:32,230 nipype.interface INFO:
	 stderr 2020-03-31T07:58:32.228653:++ 1 Blocks * 3 polynomials -- 3 polort regressors
200331-07:58:32,230 nipype.interface INFO:
	 stderr 2020-03-31T07:58:32.228653: + -- 6 other fixed ort regressors
200331-07:58:32,231 nipype.interface INFO:
	 stderr 2020-03-31T07:58:32.228653:++ 819 retained time points MINUS 21 regressors ==> 798 D.O.F. left
200331-07:58:32,232 nipype.interface INFO:
	 stderr 2020-03-31T07:58:32.232034:++ no -mask option ==> processing all 325325 voxels in dataset
200331-07:58:32,232 nipype.interface INFO:
	 stderr 2020-03-31T07:58:32.232034:++ Compute pseudo-inverse of fixe

## loop through subjects and denoise recall

In [9]:
subj_recall=pd.read_excel(analysis_dir+'subj_recall_sessions_run.xlsx')

In [10]:
for i, sub in subj_recall.iterrows():
    #load in confounds and select relevant columns
    confound_full = pd.read_csv(data_dir+sub['subj_id']+'/'+sub['ses']+'/func/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+ '_desc-confounds_regressors.tsv', sep='\t')
    index = ['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']
    confound_motion_outliers = confound_full.filter(regex="motion_outlier")
    confound_motion = confound_full[index]
    confound_select = pd.concat([confound_motion_outliers,confound_motion],axis = 1)
    confound_select.to_csv(data_dir+sub['subj_id']+'/'+sub['ses']+'/func/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+ '_desc-confounds_regressors_select.1D', header=False, index= False)
    
    #perform filtering and denoising
    tproject = afni.TProject()
    tproject.inputs.in_file = data_dir+sub['subj_id']+'/'+sub['ses']+'/func/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+'_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
    tproject.inputs.bandpass = (0.005, 99999)
    tproject.inputs.ort = (data_dir+sub['subj_id']+'/'+sub['ses']+'/func/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+ '_desc-confounds_regressors_select.1D')
    tproject.inputs.automask = False
    tproject.inputs.out_file = analysis_dir+'confound_corrected_data/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+'_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised.nii.gz'
    tproject.inputs.norm = False
    res = tproject.run() 

200331-09:03:56,140 nipype.interface INFO:
	 stderr 2020-03-31T09:03:56.140166:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200331-09:03:56,142 nipype.interface INFO:
	 stderr 2020-03-31T09:03:56.140166:++ Authored by: Cox the Algebraic (Linear)
200331-09:03:56,144 nipype.interface INFO:
	 stderr 2020-03-31T09:03:56.144831:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_output_test/fmriprep/sub-000/ses-01/func/sub-000_ses-01_task-spokenrecall_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
200331-09:03:56,145 nipype.interface INFO:
	 stderr 2020-03-31T09:03:56.144831:     Warnings of this type will be muted for this session.
200331-09:03:56,146 nipype.interface INFO:
	 stderr 2020-03-31T09:03:56.144831:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
200331-09:04:01,378 nipype.interface INFO:
	 stderr 2020-03-31T09:04:01.378322:++ Setting up regressors
200331-09:04:01,378 

200331-09:04:49,182 nipype.interface INFO:
	 stderr 2020-03-31T09:04:49.181553:** ERROR: dataset NOT written to disk!
200331-09:04:49,182 nipype.interface INFO:
	 stderr 2020-03-31T09:04:49.181553:++ ===== clock time = 14s 611ms
200331-09:04:49,373 nipype.interface INFO:
	 stderr 2020-03-31T09:04:49.372977:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200331-09:04:49,374 nipype.interface INFO:
	 stderr 2020-03-31T09:04:49.372977:++ Authored by: Cox the Algebraic (Linear)
200331-09:04:49,394 nipype.interface INFO:
	 stderr 2020-03-31T09:04:49.394585:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_output_test/fmriprep/sub-102/ses-02/func/sub-102_ses-02_task-spokenrecall_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
200331-09:04:49,395 nipype.interface INFO:
	 stderr 2020-03-31T09:04:49.394585:     Warnings of this type will be muted for this session.
200331-09:04:49,396 nipype.interface INFO:
	

200331-09:05:09,216 nipype.interface INFO:
	 stderr 2020-03-31T09:05:09.215844:** ERROR: dataset NOT written to disk!
200331-09:05:09,217 nipype.interface INFO:
	 stderr 2020-03-31T09:05:09.215844:++ ===== clock time = 9s 400ms
200331-09:05:09,928 nipype.interface INFO:
	 stderr 2020-03-31T09:05:09.928059:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200331-09:05:09,929 nipype.interface INFO:
	 stderr 2020-03-31T09:05:09.928059:++ Authored by: Cox the Algebraic (Linear)
200331-09:05:09,949 nipype.interface INFO:
	 stderr 2020-03-31T09:05:09.949124:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_output_test/fmriprep/sub-105/ses-02/func/sub-105_ses-02_task-spokenrecall_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
200331-09:05:09,950 nipype.interface INFO:
	 stderr 2020-03-31T09:05:09.949124:     Warnings of this type will be muted for this session.
200331-09:05:09,950 nipype.interface INFO:
	 

200331-09:05:41,727 nipype.interface INFO:
	 stderr 2020-03-31T09:05:41.726481:** ERROR: dataset NOT written to disk!
200331-09:05:41,727 nipype.interface INFO:
	 stderr 2020-03-31T09:05:41.726481:++ ===== clock time = 6s 89ms
200331-09:05:41,867 nipype.interface INFO:
	 stderr 2020-03-31T09:05:41.867794:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200331-09:05:41,869 nipype.interface INFO:
	 stderr 2020-03-31T09:05:41.867794:++ Authored by: Cox the Algebraic (Linear)
200331-09:05:41,870 nipype.interface INFO:
	 stderr 2020-03-31T09:05:41.870420:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_output_test/fmriprep/sub-108/ses-02/func/sub-108_ses-02_task-spokenrecall_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
200331-09:05:41,870 nipype.interface INFO:
	 stderr 2020-03-31T09:05:41.870420:     Warnings of this type will be muted for this session.
200331-09:05:41,871 nipype.interface INFO:
	 s

200331-09:06:12,77 nipype.interface INFO:
	 stderr 2020-03-31T09:06:12.077069:** ERROR: dataset NOT written to disk!
200331-09:06:12,78 nipype.interface INFO:
	 stderr 2020-03-31T09:06:12.077069:++ ===== clock time = 18s 310ms
200331-09:06:12,433 nipype.interface INFO:
	 stderr 2020-03-31T09:06:12.433147:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200331-09:06:12,435 nipype.interface INFO:
	 stderr 2020-03-31T09:06:12.433147:++ Authored by: Cox the Algebraic (Linear)
200331-09:06:12,457 nipype.interface INFO:
	 stderr 2020-03-31T09:06:12.457903:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_output_test/fmriprep/sub-101/ses-02/func/sub-101_ses-02_task-spokenrecall_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
200331-09:06:12,460 nipype.interface INFO:
	 stderr 2020-03-31T09:06:12.457903:     Warnings of this type will be muted for this session.
200331-09:06:12,461 nipype.interface INFO:
	 s

...

200331-09:07:44,955 nipype.interface INFO:
	 stderr 2020-03-31T09:07:44.955589:++ Starting project-orization
200331-09:07:45,645 nipype.interface INFO:
	 stderr 2020-03-31T09:07:45.645725:++ Convert results to output dataset
200331-09:07:47,574 nipype.interface INFO:
	 stderr 2020-03-31T09:07:47.574448:** ERROR: output dataset name 'sub-109_ses-02_task-spokenrecall_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised.nii.gz' conflicts with existing file
200331-09:07:47,575 nipype.interface INFO:
	 stderr 2020-03-31T09:07:47.574448:** ERROR: dataset NOT written to disk!
200331-09:07:47,575 nipype.interface INFO:
	 stderr 2020-03-31T09:07:47.574448:++ ===== clock time = 5s 154ms
200331-09:07:47,725 nipype.interface INFO:
	 stderr 2020-03-31T09:07:47.725595:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200331-09:07:47,727 nipype.interface INFO:
	 stderr 2020-03-31T09:07:47.725595:++ Authored by: Cox the Algebraic (Linear)
200331-09:07:47,755 nipype.interface INFO:


200331-09:08:10,136 nipype.interface INFO:
	 stderr 2020-03-31T09:08:10.136541:++ Starting project-orization
200331-09:08:11,286 nipype.interface INFO:
	 stderr 2020-03-31T09:08:11.286715:++ Convert results to output dataset
200331-09:08:24,891 nipype.interface INFO:
	 stderr 2020-03-31T09:08:24.891385:++ Output dataset /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/SRM/confound_corrected_data/sub-117_ses-02_task-spokenrecall_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised.nii.gz
200331-09:08:24,892 nipype.interface INFO:
	 stderr 2020-03-31T09:08:24.891385:++ ===== clock time = 18s 114ms
200331-09:08:25,94 nipype.interface INFO:
	 stderr 2020-03-31T09:08:25.094179:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200331-09:08:25,96 nipype.interface INFO:
	 stderr 2020-03-31T09:08:25.094179:++ Authored by: Cox the Algebraic (Linear)
200331-09:08:25,117 nipype.interface INFO:
	 stderr 2020-03-31T09:08:25.117930:** AFNI converts NIFTI_datatype=4 (INT16) in file /Use

200331-09:08:51,727 nipype.interface INFO:
	 stderr 2020-03-31T09:08:51.727148:++ Starting project-orization
200331-09:08:52,93 nipype.interface INFO:
	 stderr 2020-03-31T09:08:52.093737:++ Convert results to output dataset
200331-09:09:01,656 nipype.interface INFO:
	 stderr 2020-03-31T09:09:01.656147:++ Output dataset /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/SRM/confound_corrected_data/sub-121_ses-02_task-spokenrecall_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised.nii.gz
200331-09:09:01,656 nipype.interface INFO:
	 stderr 2020-03-31T09:09:01.656147:++ ===== clock time = 12s 70ms
200331-09:09:01,792 nipype.interface INFO:
	 stderr 2020-03-31T09:09:01.791951:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200331-09:09:01,793 nipype.interface INFO:
	 stderr 2020-03-31T09:09:01.791951:++ Authored by: Cox the Algebraic (Linear)
200331-09:09:01,811 nipype.interface INFO:
	 stderr 2020-03-31T09:09:01.811579:** AFNI converts NIFTI_datatype=4 (INT16) in file /Use

200331-09:09:30,923 nipype.interface INFO:
	 stderr 2020-03-31T09:09:30.923080:++ ===== clock time = 7s 18ms
200331-09:09:31,121 nipype.interface INFO:
	 stderr 2020-03-31T09:09:31.121088:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200331-09:09:31,122 nipype.interface INFO:
	 stderr 2020-03-31T09:09:31.121088:++ Authored by: Cox the Algebraic (Linear)
200331-09:09:31,157 nipype.interface INFO:
	 stderr 2020-03-31T09:09:31.157913:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_output_test/fmriprep/sub-117/ses-02/func/sub-117_ses-02_task-spokenrecall_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
200331-09:09:31,158 nipype.interface INFO:
	 stderr 2020-03-31T09:09:31.157913:     Warnings of this type will be muted for this session.
200331-09:09:31,158 nipype.interface INFO:
	 stderr 2020-03-31T09:09:31.157913:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
200331-09:09:32

200331-09:10:12,152 nipype.interface INFO:
	 stderr 2020-03-31T09:10:12.151981:++ ===== clock time = 12s 40ms
200331-09:10:12,320 nipype.interface INFO:
	 stderr 2020-03-31T09:10:12.319932:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
200331-09:10:12,321 nipype.interface INFO:
	 stderr 2020-03-31T09:10:12.319932:++ Authored by: Cox the Algebraic (Linear)
200331-09:10:12,329 nipype.interface INFO:
	 stderr 2020-03-31T09:10:12.329889:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_output_test/fmriprep/sub-121/ses-02/func/sub-121_ses-02_task-spokenrecall_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
200331-09:10:12,331 nipype.interface INFO:
	 stderr 2020-03-31T09:10:12.329889:     Warnings of this type will be muted for this session.
200331-09:10:12,332 nipype.interface INFO:
	 stderr 2020-03-31T09:10:12.329889:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
200331-09:10:1

## calculate tSNR

In [23]:
from nipype.algorithms.confounds import TSNR

In [26]:
tsnr = TSNR()
tsnr.inputs.in_file = data_dir+'sub-000/ses-01/func/sub-000_ses-01_task-movieviewing_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii'
res = tsnr.run()

In [10]:
for i, sub in subj_recall.iterrows():
    tsnr = TSNR()
    tsnr.inputs.in_file = data_dir+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+'_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised.nii.gz'
    res = tsnr.run() 

NameError: name 'TSNR' is not defined